In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
file_list = drive.ListFile({'q': "'1iW1GMIWmDm6h6uGxJPjk2bt-7JcQJETM' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: OCR.ipynb, id: 1_DYFu440zgSozaS2W5i7LH018UQU-C1y
title: xml, id: 12UyuTf3rN2lQCOfCb5femi3A_q37eCy2
title: emnist-balanced-train.csv, id: 1HfeTC65KaW3FOE2httDR6wYySvdUbF4o
title: emnist-balanced-test.csv, id: 12dgVBkW6GCa-1dQkSMrLQ55ee-FUgv3M


In [0]:
train_downloaded = drive.CreateFile({'id': '1HfeTC65KaW3FOE2httDR6wYySvdUbF4o'})
train_downloaded.GetContentFile('train.csv')
test_downloaded = drive.CreateFile({'id': '12dgVBkW6GCa-1dQkSMrLQ55ee-FUgv3M'})
test_downloaded.GetContentFile('test.csv')

In [5]:
import numpy as np
import keras
import matplotlib.pyplot as plt
import pandas as pd
import os

Using TensorFlow backend.


In [6]:
data_train = pd.read_csv("train.csv").values
data_test = pd.read_csv("test.csv").values

X_train = data_train[:, 1:]
Y_train = data_train[:,0]
X_test = data_test[:, 1:]
Y_test = data_test[:,0]

X_train = np.reshape(X_train, (X_train.shape[0], 28, 28, 1)) / 255
X_test = np.reshape(X_test, (X_test.shape[0], 28, 28, 1)) / 255
Y_train = np.reshape(Y_train, (Y_train.shape[0], 1))
Y_test = np.reshape(Y_test, (Y_test.shape[0], 1))

Y_train = keras.utils.to_categorical(Y_train, 47)
Y_test = keras.utils.to_categorical(Y_test, 47)

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(112799, 28, 28, 1)
(112799, 47)
(18799, 28, 28, 1)
(18799, 47)


In [0]:
def make_model(input_shape):
  X_input = keras.layers.Input(input_shape)
  X = keras.layers.Conv2D(32, (3,3), padding = 'same')(X_input)
  X = keras.layers.BatchNormalization(axis = 3)(X)
  X = keras.layers.Activation('relu')(X)
  
  X = keras.layers.MaxPooling2D((2,2))(X)
  
  X = keras.layers.Conv2D(64, (3,3), padding = 'same')(X)
  X = keras.layers.BatchNormalization(axis = 3)(X)
  X = keras.layers.Activation('relu')(X)
  
  X = keras.layers.MaxPooling2D((2,2))(X)
  
  X = keras.layers.Conv2D(128, (4,4))(X)
  X = keras.layers.BatchNormalization(axis = 3)(X)
  X = keras.layers.Activation('relu')(X)
  
  X = keras.layers.Flatten()(X)
  X = keras.layers.Dropout(0.5)(X)
  X = keras.layers.Dense(256, activation = 'relu')(X)
  X = keras.layers.Dropout(0.5)(X)
  X = keras.layers.Dense(47, activation = 'softmax')(X)
  
  model = keras.models.Model(inputs = X_input, outputs = X, name = "model")
  model.summary()
  
  return model

In [8]:
model = make_model((28, 28, 1))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 14, 64)        256       
__________

In [0]:
opt = keras.optimizers.Adam(lr = 0.0001)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [12]:
model.fit(X_train, Y_train, batch_size = 512, epochs = 20, validation_data = (X_test, Y_test))

Train on 112799 samples, validate on 18799 samples
Epoch 1/20
112799/112799 [==============================] - 19s 165us/step - loss: 0.2553 - acc: 0.9015 - val_loss: 0.3017 - val_acc: 0.8957
Epoch 2/20
112799/112799 [==============================] - 17s 150us/step - loss: 0.2499 - acc: 0.9034 - val_loss: 0.3030 - val_acc: 0.8953
Epoch 3/20
112799/112799 [==============================] - 17s 149us/step - loss: 0.2467 - acc: 0.9037 - val_loss: 0.2950 - val_acc: 0.8948
Epoch 4/20
112799/112799 [==============================] - 17s 150us/step - loss: 0.2426 - acc: 0.9052 - val_loss: 0.3172 - val_acc: 0.8901
Epoch 5/20
112799/112799 [==============================] - 17s 151us/step - loss: 0.2419 - acc: 0.9051 - val_loss: 0.2946 - val_acc: 0.8960
Epoch 6/20
112799/112799 [==============================] - 17s 150us/step - loss: 0.2398 - acc: 0.9058 - val_loss: 0.2976 - val_acc: 0.8954
Epoch 7/20
112799/112799 [==============================] - 17s 151us/step - loss: 0.2352 - acc: 0.9077